In [1]:
import re
import torch
import pandas as pd
from googletrans import Translator
from transformers import AutoTokenizer
from transformers.models.xlm_roberta.tokenization_xlm_roberta_fast import XLMRobertaTokenizerFast

In [2]:
translator = Translator()
translator.translate("Невероятные приключения", src="ru", dest="en").text

'Incredible adventures'

In [3]:
df = pd.read_csv("top_250_films_reviews_filtered.csv")
df.head()

,film_id,review_id,type,date,pos,neg,author,title,description
0,435,65581,POSITIVE,2007-06-26T19:47:00,485,63,P.S.SeLeNa,NaN,«Они помогли ему убить себя и так происходит к...
1,435,151932,POSITIVE,2008-01-25T03:08:00,256,26,Боб,Дорога длинною в жизнь,"«Боже, иногда Зеленая миля так длинна… «. «М..."
2,435,72099,NEUTRAL,2007-05-23T23:34:00,209,24,КиноПоиск,NaN,"I. Два творца: Стивен Кинг. Имя, не нуждающеес..."
3,435,114905,POSITIVE,2007-10-21T18:32:00,175,20,Exitlude,NaN,Они помогли ему убить себя. «Они помогли ему у...
4,435,1715054,NEGATIVE,2013-02-06T17:48:00,148,375,Maria_M,"Тот момент, когда десяток отрицательный коммен...","Посмотреть «Зеленую милю» меня заставило, как ..."


In [17]:
MODEL_MAX_LENGTH = 512 # 514
model_name = 'unicamp-dl/mMiniLM-L6-v2-mmarco-v2'
tokenizer: XLMRobertaTokenizerFast  = AutoTokenizer.from_pretrained(model_name)
tokenizer.model_max_length = MODEL_MAX_LENGTH

In [18]:
def truncate(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    output = tokenizer.decode(inputs["input_ids"][0]).lstrip("<s> ").rstrip("</s>")
    return output

In [22]:
data = df.iloc[1]["description"]
print(data)
truncate(data)

 «Боже, иногда Зеленая миля так длинна… «.  «Мы все заслужили смерть, все, без исключения. Но ведь тот же Бог так жестоко принес в жертву Джона Коффи, пытавшегося творить добро по-своему, как только ветхозаветный пророк мог принести в жертву беззащитного агнца, …как Авраам пожертвовал бы собственным сыном, если бы ему и впрямь повелели. Я вспоминаю слова Джона, что Уортон убил близняшек Деттерик их любовью друг к другу, и такое происходит каждый день по всему миру. Значит, Бог позволяет, чтобы оно случалось, и, когда мы говорим: «Я не понимаю», Бог отвечает: «Мне все равно». Стивен Кинг («Зеленая миля»). О, не спеши туда, где жизнь светлей и чище! А. Толстой. Сюжет идентичен с книгой Стивена Кинга, поэтому буду краток. Пол Эджкомб работает начальником тюремного блока. Это не простой блок — здесь проводят свои последние недели люди, осужденные на смертную казнь — самые отъявленные уголовники: маньяки, насильники, убийцы. Пол проработал здесь много лет, провел сотни казней и научился быт

'«Боже, иногда Зеленая миля так длинна... «. «Мы все заслужили смерть, все, без исключения. Но ведь тот же Бог так жестоко принес в жертву Джона Коффи, пытавшегося творить добро по-своему, как только ветхозаветный пророк мог принести в жертву беззащитного агнца,...как Авраам пожертвовал бы собственным сыном, если бы ему и впрямь повелели. Я вспоминаю слова Джона, что Уортон убил близняшек Деттерик их любовью друг к другу, и такое происходит каждый день по всему миру. Значит, Бог позволяет, чтобы оно случалось, и, когда мы говорим: «Я не понимаю», Бог отвечает: «Мне все равно». Стивен Кинг («Зеленая миля»). О, не спеши туда, где жизнь светлей и чище! А. Толстой. Сюжет идентичен с книгой Стивена Кинга, поэтому буду краток. Пол Эджкомб работает начальником тюремного блока. Это не простой блок — здесь проводят свои последние недели люди, осужденные на смертную казнь — самые отъявленные уголовники: маньяки, насильники, убийцы. Пол проработал здесь много лет, провел сотни казней и научился б

In [5]:
translated = translator.translate(data, dest="en", src="ru")
translated.text

'“They helped him kill himself and this happens every day all over the world” ... a film that I can safely call a masterpiece. The film that lasts three hours and which fly unnoticed. A film that looks every time in the first. A film that cannot be forgotten and which cannot be ignored. The film, which makes us think about the life that we spend on trifles, amending ourselves with the illusion of immortality, about death, which does not spare anyone. A film about evil, which is not always punishable and about good, which is sometimes powerless to change anything. And you just have to look at how the court rises, on who should not sit on the bench of the defendants. The film that changed my worldview and taught how to treat life differently. 10 out of 10'

In [32]:
class Test:
    def __init__(self):
        self.i = 0
    def __call__(self, text):
        self.i += 1
        print(f"\r{self.i}", end="")
        text = truncate(text)
        translated = translator.translate(text, dest="en", src="ru")
        return translated.text

In [33]:
test = Test()
dft = df.copy()
dft["description"] = dft["description"].apply(test)
print(len(dft))
dft.to_csv("top_250_films_reviews_filtered_en.csv", index=False)
dft

9502

,film_id,review_id,type,date,pos,neg,author,title,description
0,435,65581,POSITIVE,2007-06-26T19:47:00,485,63,P.S.SeLeNa,NaN,“They helped him kill himself and this happens...
1,435,151932,POSITIVE,2008-01-25T03:08:00,256,26,Боб,Дорога длинною в жизнь,"“God, sometimes the green mile is so long ...”..."
2,435,72099,NEUTRAL,2007-05-23T23:34:00,209,24,КиноПоиск,NaN,I. Two creators: Stephen King. A name that doe...
3,435,114905,POSITIVE,2007-10-21T18:32:00,175,20,Exitlude,NaN,"They helped him kill himself. ""They helped him..."
4,435,1715054,NEGATIVE,2013-02-06T17:48:00,148,375,Maria_M,"Тот момент, когда десяток отрицательный коммен...","Watching the Green Mile made me like many, the..."
...,...,...,...,...,...,...,...,...,...
9498,47237,597888,POSITIVE,2009-08-27T15:43:53,25,6,Christian Robson,Легенда: Начало,Rethinking the classic comic. The beginning of...
9499,47237,369940,POSITIVE,2009-01-31T21:16:39,25,6,Mexoc,Вот он какой - Бэтмен,I watched this film twice and both times this ...
9500,47237,209524,POSITIVE,2008-06-02T20:11:16,25,3,Paul Freedom,"Не предистория, а начало!","First of all, I will say that this film made m..."
9501,47237,54041,POSITIVE,2007-01-29T14:04:21,25,6,2galaxy,NaN,"Christian Bale is a real Batman, he managed to..."


In [ ]:
from easynmt import EasyNMT

In [8]:
model = EasyNMT('opus-mt')
model.device

'cuda'

In [9]:
doc = df.iloc[0]["description"]
doc

'«Они помогли ему убить себя и так происходит каждый день во всем мире»… Фильм, который я могу смело назвать шедевром. Фильм, который длится три часа и которые пролетают незаметно. Фильм, который смотрится каждый раз как в первый. Фильм, который нельзя забыть и которым нельзя не проникнуться. Фильм, котрый заставляет задуматься о жизни, которую мы тратим на пустяки, теша себя иллюзией бессмертия, о смерти, которая не щадит никого. Фильм о зле, которое не всегда наказуемо и о добре, которое подчас бессильно что-либо изменить. И тебе остается лишь смотреть на то, как вершится суд, над тем кто не должен сидеть на скамье подсудимых. Фильм, который изменил мое мировоззрение и научил по-другому относиться к жизни. 10 из 10'

In [10]:
translated = model.translate(doc, target_lang="en", source_lang="ru")
translated

'"They helped him kill himself and that\'s what happens every day in the world"... a film I can safely call a masterpiece. A film that lasts three hours and it\'s not visible. A movie that looks like it\'s the first time. A film that can\'t be forgotten and can\'t be ignored. A film that makes you think about the life we spend on nothing, the illusion of immortality, the death that spares no one. A film about evil, which is not always punishable by good, which is sometimes powerless to change anything. And all you have to do is watch the trial, the one who shouldn\'t sit on the defendant\'s bench. A film that changed my worldview and taught me a different way of dealing with life. 10 out of 10'

In [11]:
translated = model.translate(df["description"].to_list(), target_lang="en", source_lang="ru", batch_size=8, show_progress_bar=True)

  0%|          | 24/256292.0 [01:02<185:15:36,  2.60s/it]


KeyboardInterrupt: 

In [39]:
t = pd.DataFrame(data={"text": ["<i>Что ещё за Канзас Сити Шаффл? Канзас Сити Шаффл — это когда все смотрят направо, а ты идёшь налево.</i>. <i>Было время:</i>"]})
t

,text
0,<i>Что ещё за Канзас Сити Шаффл? Канзас Сити Ш...


In [43]:
t["text"] = t["text"].str.replace("<\w+>([^<>]+)<\/\w+>", "A", regex=True)
t

,text
0,A. A


In [42]:
re.findall("<\w+>([^<>]+)<\/\w+>", t.iloc[0]["text"])

['Что ещё за Канзас Сити Шаффл? Канзас Сити Шаффл — это когда все смотрят направо, а ты идёшь налево.',
 'Было время:']